In [1]:
from pymongo import MongoClient
from collections import OrderedDict
from metadataclient.api import (insert_run_start, insert_descriptor,
                                insert_run_stop, insert_event, bulk_insert_events)
from metadataclient.api import (find_run_starts, find_descriptors, find_events,
                               find_run_stops, find_last)
from metadataclient.conf import connection_config
from urllib3.exceptions import ConnectionError
import boltons

In [2]:
class MdsMigrationException(Exception):
    """Goodbye cruel world!"""
    pass

**_Ensure global connection management is functional_**

In [3]:
try:
    MONGO_HOST = 'kusadasi'
    connection_config['host'] = MONGO_HOST 
    res = next(find_last())
except Exception:
    pass

**Connection Management**

In [4]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MIGRATION_DB = 'datastore2'
# assuming both mongo and tornado server on the same machine
connection_config['host'] = MONGO_HOST 
connection_config['port'] = 7770
pymongo_client = MongoClient(MONGO_HOST, MONGO_PORT)
database = pymongo_client[MIGRATION_DB]
try:
    res = (next(find_last()))
except Exception:
    raise MdsMigrationException('Seems like metadataservice is not accessible')

**Get all run_starts**

In [5]:
print(database)

rstart_crsr = database.run_start.find()

Database(MongoClient('localhost', 27017), 'datastore2')


In [6]:
run_starts = OrderedDict()
for rstart in rstart_crsr:
    run_starts[rstart['uid']] = rstart
    bcfg_crsr = database.beamline_config.find()
    for cfg in bcfg_crsr:
        rstart['config'] = cfg

**Get all descriptors and insert them using metadataclient**

In [7]:
def rstart_oid_lookup(database, oid):
    try:
        next(database.run_start.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like run_start with this oid does not exist')

In [8]:
def descriptor_oid_lookup(database, oid):
    try:
        next(database.event_descriptor.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like descriptor with this oid does not exist')

In [9]:
desc_crsr = database.event_descriptor.find()
descriptors = OrderedDict()
for desc in desc_crsr:
    rs_oid = desc['run_start']
    rstart = rstart_oid_lookup(database, rs_oid)
    # make sure run_start exists
    desc['run_start'] = run_starts[rstart['uid']]['uid'] 
    descriptors[desc['uid']] = desc

MdsMigrationException: Seems like run_start with this oid does not exist

In [10]:
event_crsr = database.event_descriptor.find()
events = OrderedDict()
for ev in event_crsr:
    rs_oid = ev['run_start']
    desc = descriptor_oid_lookup(database, rs_oid)
    # make sure run_start exists
    desc['run_start'] = descriptors[desc['uid']]['uid'] 
    events[ev['uid']] = ev

MdsMigrationException: Seems like descriptor with this oid does not exist

In [11]:
print(run_starts)

OrderedDict([('9d3b3e33-f192-4015-b234-8b01a7c3aac4', {'beamline_id': 'example', 'time': 1451488543.479466, 'uid': '9d3b3e33-f192-4015-b234-8b01a7c3aac4', 'scan_id': 105, 'group': '', 'custom': {}, 'owner': 'stepper', 'project': '', '_id': ObjectId('5683f51ffa448316734d803a')}), ('85cfae0c-1226-4097-9608-8deb3e3cb4c2', {'beamline_id': 'example', 'time': 1451488544.181346, 'uid': '85cfae0c-1226-4097-9608-8deb3e3cb4c2', 'scan_id': 1, 'group': '', 'custom': {}, 'owner': 'docbrown', 'project': '', '_id': ObjectId('5683f520fa448316734d8040')}), ('1bf2f071-5a89-4f5b-a33e-d2f0b55d5856', {'beamline_id': 'example', 'time': 1451488551.831704, 'uid': '1bf2f071-5a89-4f5b-a33e-d2f0b55d5856', 'scan_id': 2, 'group': '', 'custom': {}, 'owner': 'docbrown', 'project': '', '_id': ObjectId('5683f527fa448316734d8139')}), ('9471cdc0-9808-43da-b46c-33441cefe330', {'beamline_id': 'example', 'time': 1451488552.525256, 'uid': '9471cdc0-9808-43da-b46c-33441cefe330', 'scan_id': 3, 'group': '', 'custom': {}, 'owne